In [1]:
import torch
from torch import nn, optim
from torchvision import datasets
from torchvision.transforms import ToTensor 
from torch.utils.data import DataLoader 
from matplotlib import pyplot as plt

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
batch_size = 64

train_loader = DataLoader(training_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [6]:
classes = [ 
    "T-shirt/top",
    "Trouser",
    "Pullover"
    "Dress", 
    "Coat",
    "Sandal", 
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [7]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
device

'cuda'

In [10]:
class ClothClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
        )
    
    def forward(self, x):
        return self.network(x)

In [31]:
model = ClothClassifier().to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [32]:
epochs = 2

for epoch in range(epochs):
    for (batch, (images, labels)) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # forward
        pred = model(images)
        loss = loss_fn(pred, labels)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # update weights
        optimizer.step()
        
        if(batch % 100 == 0):
            print(f'Batch: {batch}, Loss: {loss.item()}')

Batch: 0, Loss: 2.3122611045837402
Batch: 100, Loss: 0.8135560154914856
Batch: 200, Loss: 0.43670183420181274
Batch: 300, Loss: 0.6382991075515747
Batch: 400, Loss: 0.5522520542144775
Batch: 500, Loss: 0.4713236689567566
Batch: 600, Loss: 0.4220947027206421
Batch: 700, Loss: 0.6159631013870239
Batch: 800, Loss: 0.5441025495529175
Batch: 900, Loss: 0.5091257691383362
Batch: 0, Loss: 0.3195571005344391
Batch: 100, Loss: 0.42120999097824097
Batch: 200, Loss: 0.31393539905548096
Batch: 300, Loss: 0.4686932861804962
Batch: 400, Loss: 0.4297303259372711
Batch: 500, Loss: 0.35963350534439087
Batch: 600, Loss: 0.3722023367881775
Batch: 700, Loss: 0.5687207579612732
Batch: 800, Loss: 0.48692530393600464
Batch: 900, Loss: 0.4548996090888977


In [33]:
# Testing the model
model.eval()
# Switch to evaluation mode
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        # Append Labels and predictions to lists 
        all_labels.extend(labels.cpu().numpy()) 
        all_predicted.extend(predicted.cpu().numpy())

In [34]:
from sklearn.metrics import classification_report

report = classification_report(all_labels, all_predicted)
print(report)

              precision    recall  f1-score   support

           0       0.80      0.84      0.82      1000
           1       0.95      0.97      0.96      1000
           2       0.85      0.59      0.69      1000
           3       0.86      0.86      0.86      1000
           4       0.64      0.89      0.74      1000
           5       0.96      0.94      0.95      1000
           6       0.71      0.55      0.62      1000
           7       0.89      0.97      0.93      1000
           8       0.94      0.97      0.95      1000
           9       0.97      0.91      0.94      1000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

